## Iteration 3

In [1]:
import pandas as pd
import numpy as np
import json
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from datetime import datetime, timedelta

warnings.filterwarnings('ignore')

## With basic Imputation

In [2]:
# ============================================================================
# LOAD DATA
# ============================================================================
print("="*80)
print("MODEL D: LASSO-SELECTED EQUATION EXTRACTION")
print("="*80)

data_path = Path().cwd().parent.parent / 'data' / 'dataprocessed'
df = pd.read_csv(data_path / 'lagged_predictive_dataset.csv')

with open(data_path / 'lag_selection_metadata.json', 'r') as f:
    metadata = json.load(f)

df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)
y = df[metadata['focal_column']]

# Remove focal lags
focal_lag_cols = [col for col in df.columns if 'base_rate_lag' in col.lower()]
df = df.drop(columns=focal_lag_cols)

# Create week features
df['week_of_year'] = df['date'].dt.isocalendar().week.astype(float)
week_mean = df['week_of_year'].mean()
df['week_centered'] = df['week_of_year'] - week_mean
df['week_squared'] = df['week_centered'] ** 2
df['week_cubic'] = df['week_centered'] ** 3
df['week_quartic'] = df['week_centered'] ** 4
df['sin_week'] = np.sin(2 * np.pi * df['week_of_year'] / 52).astype(float)
df['cos_week'] = np.cos(2 * np.pi * df['week_of_year'] / 52).astype(float)

# Seasonal indicators
if 'is_holiday' not in df.columns:
    df['is_holiday'] = 0
    df.loc[df['date'].dt.month.isin([12, 1]), 'is_holiday'] = 1
    df.loc[(df['date'].dt.month == 7) & (df['date'].dt.day <= 7), 'is_holiday'] = 1
    df.loc[(df['date'].dt.month == 11) & (df['date'].dt.day >= 22), 'is_holiday'] = 1

df['is_peak_season'] = ((df['date'].dt.month.isin([6, 7, 8])) | 
                        (df['date'].dt.month.isin([12, 1]))).astype(int)
df['is_summer'] = df['date'].dt.month.isin([6, 7, 8]).astype(int)

# Get competitors
competitor_cols = [col for col in df.columns if 'booking-us' in col and 'lag' in col]
comp_corr_with_target = df[competitor_cols].corrwith(y).abs().sort_values(ascending=False)

all_competitors = []
for comp in comp_corr_with_target.index:
    is_redundant = False
    for existing in all_competitors:
        if abs(df[comp].corr(df[existing])) > 0.90:
            is_redundant = True
            break
    if not is_redundant:
        all_competitors.append(comp)

seasonal_cols = ['is_holiday', 'is_peak_season', 'is_summer']

# Prepare feature pool for Lasso
features_A = all_competitors + seasonal_cols + ['sin_week', 'cos_week']
features_B = all_competitors + seasonal_cols + ['week_centered', 'week_squared', 'week_cubic', 'week_quartic']
features_D = list(set(features_A + features_B))

# ============================================================================
# RUN LASSO SELECTION
# ============================================================================
print("\nRunning Lasso feature selection...")

X_D = df[features_D].dropna()
y_D = y[X_D.index]

scaler = StandardScaler()
X_D_scaled = scaler.fit_transform(X_D)

lasso = LassoCV(cv=5, random_state=42, max_iter=10000, n_jobs=-1)
lasso.fit(X_D_scaled, y_D)

selected_mask = lasso.coef_ != 0
selected_features = np.array(features_D)[selected_mask]
n_selected = selected_mask.sum()

print(f"Lasso selected {n_selected} features from {len(features_D)}")
print(f"Optimal alpha: {lasso.alpha_:.6f}")

# ============================================================================
# REFIT WITH OLS
# ============================================================================
print("\nRefitting with OLS on selected features...")

X_selected = df[list(selected_features)].dropna()
y_selected = y[X_selected.index]

X_with_const = sm.add_constant(X_selected)
model_D = sm.OLS(y_selected, X_with_const).fit(cov_type='HC1')

print(f"Model D: Adj R² = {model_D.rsquared_adj:.4f}")
print(f"Significant features: {(model_D.pvalues[1:] < 0.05).sum()}/{n_selected}")

# ============================================================================
# CALCULATE RELATIVE ERROR METRICS (NEW SECTION)
# ============================================================================
print("\n" + "="*80)
print("RELATIVE ERROR METRICS")
print("="*80)

# Get predictions
y_pred = model_D.predict(X_with_const)

# Calculate percentage errors
percentage_errors = np.abs((y_selected - y_pred) / y_selected) * 100

# Summary statistics
mape = percentage_errors.mean()
median_ape = percentage_errors.median()
rmse = np.sqrt(model_D.mse_resid)
avg_base_rate = y_selected.mean()
rmse_as_pct = (rmse / avg_base_rate) * 100

print(f"\nAbsolute Error Metrics:")
print(f"  RMSE: ${rmse:.2f}")
print(f"  Average base_rate: ${avg_base_rate:.2f}")
print(f"  RMSE as % of average: {rmse_as_pct:.2f}%")

print(f"\nRelative Error Metrics:")
print(f"  Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"  Median Absolute Percentage Error: {median_ape:.2f}%")

print(f"\nPercentage Error Distribution:")
print(f"  25th percentile: {percentage_errors.quantile(0.25):.2f}%")
print(f"  50th percentile: {percentage_errors.quantile(0.50):.2f}%")
print(f"  75th percentile: {percentage_errors.quantile(0.75):.2f}%")
print(f"  90th percentile: {percentage_errors.quantile(0.90):.2f}%")
print(f"  95th percentile: {percentage_errors.quantile(0.95):.2f}%")

print(f"\nDays with error < 5%: {(percentage_errors < 5).sum()} ({(percentage_errors < 5).sum()/len(percentage_errors)*100:.1f}%)")
print(f"Days with error < 10%: {(percentage_errors < 10).sum()} ({(percentage_errors < 10).sum()/len(percentage_errors)*100:.1f}%)")
print(f"Days with error > 20%: {(percentage_errors > 20).sum()} ({(percentage_errors > 20).sum()/len(percentage_errors)*100:.1f}%)")

# ============================================================================
# OUTPUT FULL EQUATION
# ============================================================================
print("\n" + "="*80)
print("MODEL D: COMPLETE REGRESSION EQUATION")
print("="*80)

# Get intercept
intercept = model_D.params['const']
print(f"\nbase_rate = {intercept:.4f}")

# Categorize features
temporal_features = [f for f in selected_features if 'week' in f.lower() or 'cos' in f.lower() or 'sin' in f.lower()]
competitor_features = [f for f in selected_features if 'lag' in f.lower()]
seasonal_features = [f for f in selected_features if f in seasonal_cols]

# Print temporal features
if temporal_features:
    print("\n  # Temporal Features")
    for feat in temporal_features:
        coef = model_D.params[feat]
        pval = model_D.pvalues[feat]
        sign = "+" if coef >= 0 else "-"
        stars = '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else ''
        print(f"            {sign} {abs(coef):.6f} × {feat} {stars}")

# Print seasonal features
if seasonal_features:
    print("\n  # Seasonal Indicators")
    for feat in seasonal_features:
        coef = model_D.params[feat]
        pval = model_D.pvalues[feat]
        sign = "+" if coef >= 0 else "-"
        stars = '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else ''
        print(f"            {sign} {abs(coef):.6f} × {feat} {stars}")

# Print ALL competitor features (don't summarize)
if competitor_features:
    print("\n  # Competitor Lags (All 10 features)")
    # Sort competitors by absolute coefficient for readability
    comp_coefs = {feat: model_D.params[feat] for feat in competitor_features}
    sorted_comps = sorted(comp_coefs.items(), key=lambda x: abs(x[1]), reverse=True)
    
    for feat, coef in sorted_comps:
        pval = model_D.pvalues[feat]
        sign = "+" if coef >= 0 else "-"
        stars = '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else ''
        print(f"            {sign} {abs(coef):.6f} × {feat} {stars}")

print("\nNote: *** p<0.001, ** p<0.01, * p<0.05")

# ============================================================================
# SUMMARY TABLE
# ============================================================================
print("\n" + "="*80)
print("SELECTED FEATURES SUMMARY")
print("="*80)

summary_data = []
for feat in selected_features:
    coef = model_D.params[feat]
    se = model_D.bse[feat]
    pval = model_D.pvalues[feat]
    
    # Categorize
    if 'week' in feat.lower() or 'cos' in feat.lower() or 'sin' in feat.lower():
        category = 'Temporal'
    elif 'lag' in feat.lower():
        category = 'Competitor'
    else:
        category = 'Seasonal'
    
    summary_data.append({
        'Feature': feat,
        'Category': category,
        'Coefficient': round(coef, 6),
        'Std_Error': round(se, 6),
        'p_value': round(pval, 4),
        'Significant': '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else ''
    })

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values(['Category', 'p_value'])

print("\n" + summary_df.to_string(index=False))

print("\n" + "="*80)
print("EQUATION EXTRACTION COMPLETE")
print("="*80)
print(f"\nIntercept: ${model_D.params['const']:.2f}")
print(f"Total features: {n_selected}")
print(f"Temporal: {len(temporal_features)}")
print(f"Seasonal: {len(seasonal_features)}")
print(f"Competitors: {len(competitor_features)}")

MODEL D: LASSO-SELECTED EQUATION EXTRACTION

Running Lasso feature selection...
Lasso selected 13 features from 25
Optimal alpha: 0.443524

Refitting with OLS on selected features...
Model D: Adj R² = 0.5833
Significant features: 6/13

RELATIVE ERROR METRICS

Absolute Error Metrics:
  RMSE: $22.02
  Average base_rate: $267.61
  RMSE as % of average: 8.23%

Relative Error Metrics:
  Mean Absolute Percentage Error (MAPE): 6.36%
  Median Absolute Percentage Error: 5.23%

Percentage Error Distribution:
  25th percentile: 1.93%
  50th percentile: 5.23%
  75th percentile: 9.11%
  90th percentile: 14.85%
  95th percentile: 18.66%

Days with error < 5%: 172 (47.8%)
Days with error < 10%: 287 (79.7%)
Days with error > 20%: 13 (3.6%)

MODEL D: COMPLETE REGRESSION EQUATION

base_rate = 3.7342

  # Temporal Features
            - 0.001333 × week_cubic ***
            + 31.812072 × cos_week ***

  # Seasonal Indicators
            + 20.008728 × is_peak_season ***

  # Competitor Lags (All 10 featur

## Advanced Imputations

In [3]:
# # ============================================================================
# # LOAD DATA
# # ============================================================================
# print("="*80)
# print("MODEL D: LASSO-SELECTED EQUATION EXTRACTION")
# print("="*80)

# data_path = Path().cwd().parent.parent / 'data' / 'dataprocessed'
# df = pd.read_csv(data_path / 'lagged_predictive_dataset_advanced_imputation.csv')

# with open(data_path / 'lag_selection_metadata_advanced_imputation.json', 'r') as f:
#     metadata = json.load(f)

# df['date'] = pd.to_datetime(df['date'])
# df = df.sort_values('date').reset_index(drop=True)
# y = df[metadata['focal_column']]

# # Remove focal lags
# focal_lag_cols = [col for col in df.columns if 'base_rate_lag' in col.lower()]
# df = df.drop(columns=focal_lag_cols)

# # Create week features
# df['week_of_year'] = df['date'].dt.isocalendar().week.astype(float)
# week_mean = df['week_of_year'].mean()
# df['week_centered'] = df['week_of_year'] - week_mean
# df['week_squared'] = df['week_centered'] ** 2
# df['week_cubic'] = df['week_centered'] ** 3
# df['week_quartic'] = df['week_centered'] ** 4
# df['sin_week'] = np.sin(2 * np.pi * df['week_of_year'] / 52).astype(float)
# df['cos_week'] = np.cos(2 * np.pi * df['week_of_year'] / 52).astype(float)

# # Seasonal indicators
# if 'is_holiday' not in df.columns:
#     df['is_holiday'] = 0
#     df.loc[df['date'].dt.month.isin([12, 1]), 'is_holiday'] = 1
#     df.loc[(df['date'].dt.month == 7) & (df['date'].dt.day <= 7), 'is_holiday'] = 1
#     df.loc[(df['date'].dt.month == 11) & (df['date'].dt.day >= 22), 'is_holiday'] = 1

# df['is_peak_season'] = ((df['date'].dt.month.isin([6, 7, 8])) | 
#                         (df['date'].dt.month.isin([12, 1]))).astype(int)
# df['is_summer'] = df['date'].dt.month.isin([6, 7, 8]).astype(int)

# # Get competitors
# competitor_cols = [col for col in df.columns if 'booking-us' in col and 'lag' in col]
# comp_corr_with_target = df[competitor_cols].corrwith(y).abs().sort_values(ascending=False)

# all_competitors = []
# for comp in comp_corr_with_target.index:
#     is_redundant = False
#     for existing in all_competitors:
#         if abs(df[comp].corr(df[existing])) > 0.90:
#             is_redundant = True
#             break
#     if not is_redundant:
#         all_competitors.append(comp)

# seasonal_cols = ['is_holiday', 'is_peak_season', 'is_summer']

# # Prepare feature pool for Lasso
# features_A = all_competitors + seasonal_cols + ['sin_week', 'cos_week']
# features_B = all_competitors + seasonal_cols + ['week_centered', 'week_squared', 'week_cubic', 'week_quartic']
# features_D = list(set(features_A + features_B))

# # ============================================================================
# # RUN LASSO SELECTION
# # ============================================================================
# print("\nRunning Lasso feature selection...")

# X_D = df[features_D].dropna()
# y_D = y[X_D.index]

# scaler = StandardScaler()
# X_D_scaled = scaler.fit_transform(X_D)

# lasso = LassoCV(cv=5, random_state=42, max_iter=10000, n_jobs=-1)
# lasso.fit(X_D_scaled, y_D)

# selected_mask = lasso.coef_ != 0
# selected_features = np.array(features_D)[selected_mask]
# n_selected = selected_mask.sum()

# print(f"Lasso selected {n_selected} features from {len(features_D)}")
# print(f"Optimal alpha: {lasso.alpha_:.6f}")

# # ============================================================================
# # REFIT WITH OLS
# # ============================================================================
# print("\nRefitting with OLS on selected features...")

# X_selected = df[list(selected_features)].dropna()
# y_selected = y[X_selected.index]

# X_with_const = sm.add_constant(X_selected)
# model_D = sm.OLS(y_selected, X_with_const).fit(cov_type='HC1')

# print(f"Model D: Adj R² = {model_D.rsquared_adj:.4f}")
# print(f"Significant features: {(model_D.pvalues[1:] < 0.05).sum()}/{n_selected}")

# # ============================================================================
# # CALCULATE RELATIVE ERROR METRICS (NEW SECTION)
# # ============================================================================
# print("\n" + "="*80)
# print("RELATIVE ERROR METRICS")
# print("="*80)

# # Get predictions
# y_pred = model_D.predict(X_with_const)

# # Calculate percentage errors
# percentage_errors = np.abs((y_selected - y_pred) / y_selected) * 100

# # Summary statistics
# mape = percentage_errors.mean()
# median_ape = percentage_errors.median()
# rmse = np.sqrt(model_D.mse_resid)
# avg_base_rate = y_selected.mean()
# rmse_as_pct = (rmse / avg_base_rate) * 100

# print(f"\nAbsolute Error Metrics:")
# print(f"  RMSE: ${rmse:.2f}")
# print(f"  Average base_rate: ${avg_base_rate:.2f}")
# print(f"  RMSE as % of average: {rmse_as_pct:.2f}%")

# print(f"\nRelative Error Metrics:")
# print(f"  Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
# print(f"  Median Absolute Percentage Error: {median_ape:.2f}%")

# print(f"\nPercentage Error Distribution:")
# print(f"  25th percentile: {percentage_errors.quantile(0.25):.2f}%")
# print(f"  50th percentile: {percentage_errors.quantile(0.50):.2f}%")
# print(f"  75th percentile: {percentage_errors.quantile(0.75):.2f}%")
# print(f"  90th percentile: {percentage_errors.quantile(0.90):.2f}%")
# print(f"  95th percentile: {percentage_errors.quantile(0.95):.2f}%")

# print(f"\nDays with error < 5%: {(percentage_errors < 5).sum()} ({(percentage_errors < 5).sum()/len(percentage_errors)*100:.1f}%)")
# print(f"Days with error < 10%: {(percentage_errors < 10).sum()} ({(percentage_errors < 10).sum()/len(percentage_errors)*100:.1f}%)")
# print(f"Days with error > 20%: {(percentage_errors > 20).sum()} ({(percentage_errors > 20).sum()/len(percentage_errors)*100:.1f}%)")

# # ============================================================================
# # OUTPUT FULL EQUATION
# # ============================================================================
# print("\n" + "="*80)
# print("MODEL D: COMPLETE REGRESSION EQUATION")
# print("="*80)

# # Get intercept
# intercept = model_D.params['const']
# print(f"\nbase_rate = {intercept:.4f}")

# # Categorize features
# temporal_features = [f for f in selected_features if 'week' in f.lower() or 'cos' in f.lower() or 'sin' in f.lower()]
# competitor_features = [f for f in selected_features if 'lag' in f.lower()]
# seasonal_features = [f for f in selected_features if f in seasonal_cols]

# # Print temporal features
# if temporal_features:
#     print("\n  # Temporal Features")
#     for feat in temporal_features:
#         coef = model_D.params[feat]
#         pval = model_D.pvalues[feat]
#         sign = "+" if coef >= 0 else "-"
#         stars = '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else ''
#         print(f"            {sign} {abs(coef):.6f} × {feat} {stars}")

# # Print seasonal features
# if seasonal_features:
#     print("\n  # Seasonal Indicators")
#     for feat in seasonal_features:
#         coef = model_D.params[feat]
#         pval = model_D.pvalues[feat]
#         sign = "+" if coef >= 0 else "-"
#         stars = '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else ''
#         print(f"            {sign} {abs(coef):.6f} × {feat} {stars}")

# # Print ALL competitor features (don't summarize)
# if competitor_features:
#     print("\n  # Competitor Lags (All 10 features)")
#     # Sort competitors by absolute coefficient for readability
#     comp_coefs = {feat: model_D.params[feat] for feat in competitor_features}
#     sorted_comps = sorted(comp_coefs.items(), key=lambda x: abs(x[1]), reverse=True)
    
#     for feat, coef in sorted_comps:
#         pval = model_D.pvalues[feat]
#         sign = "+" if coef >= 0 else "-"
#         stars = '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else ''
#         print(f"            {sign} {abs(coef):.6f} × {feat} {stars}")

# print("\nNote: *** p<0.001, ** p<0.01, * p<0.05")

# # ============================================================================
# # SUMMARY TABLE
# # ============================================================================
# print("\n" + "="*80)
# print("SELECTED FEATURES SUMMARY")
# print("="*80)

# summary_data = []
# for feat in selected_features:
#     coef = model_D.params[feat]
#     se = model_D.bse[feat]
#     pval = model_D.pvalues[feat]
    
#     # Categorize
#     if 'week' in feat.lower() or 'cos' in feat.lower() or 'sin' in feat.lower():
#         category = 'Temporal'
#     elif 'lag' in feat.lower():
#         category = 'Competitor'
#     else:
#         category = 'Seasonal'
    
#     summary_data.append({
#         'Feature': feat,
#         'Category': category,
#         'Coefficient': round(coef, 6),
#         'Std_Error': round(se, 6),
#         'p_value': round(pval, 4),
#         'Significant': '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else ''
#     })

# summary_df = pd.DataFrame(summary_data)
# summary_df = summary_df.sort_values(['Category', 'p_value'])

# print("\n" + summary_df.to_string(index=False))

# print("\n" + "="*80)
# print("EQUATION EXTRACTION COMPLETE")
# print("="*80)
# print(f"\nIntercept: ${model_D.params['const']:.2f}")
# print(f"Total features: {n_selected}")
# print(f"Temporal: {len(temporal_features)}")
# print(f"Seasonal: {len(seasonal_features)}")
# print(f"Competitors: {len(competitor_features)}")

MODEL D: LASSO-SELECTED EQUATION EXTRACTION

Running Lasso feature selection...
Lasso selected 11 features from 22
Optimal alpha: 1.252324

Refitting with OLS on selected features...
Model D: Adj R² = 0.6074
Significant features: 7/11

RELATIVE ERROR METRICS

Absolute Error Metrics:
  RMSE: $21.23
  Average base_rate: $267.38
  RMSE as % of average: 7.94%

Relative Error Metrics:
  Mean Absolute Percentage Error (MAPE): 6.30%
  Median Absolute Percentage Error: 4.48%

Percentage Error Distribution:
  25th percentile: 2.05%
  50th percentile: 4.48%
  75th percentile: 9.83%
  90th percentile: 13.82%
  95th percentile: 16.84%

Days with error < 5%: 188 (52.4%)
Days with error < 10%: 274 (76.3%)
Days with error > 20%: 10 (2.8%)

MODEL D: COMPLETE REGRESSION EQUATION

base_rate = 97.5832

  # Temporal Features
            + 26.375295 × cos_week ***
            - 0.619406 × week_centered ***

  # Seasonal Indicators
            + 22.481985 × is_peak_season ***
            - 2.400652 × is_hol